In [58]:
import requests
from lxml import etree, html
import json, pandas as pd
import pandasql as ps
import sys
# 메모리 초기화
sys.stdout.flush()

# 서울 날씨

In [59]:
import requests
# Class 대상
# 기상청 기상자료개발포털 default_path
default_path = 'https://data.kma.go.kr/apiData/getData?'
sep = '&' #separator
api_type = 'type=json' # xml
data_type = 'dataCd=ASOS' # ASOS
date_type = 'dateCd=DAY'  # days
start_date = 'startDt=20160101' # start
end_date = 'endDt=20190430'     # end
station = 'stnIds=108'          # seoul
cnt = 'schListCnt=999'          # page by (max) 
# cnt = 'schListCnt=10'          # page by (max) 
page = 'pageIndex='            # page by count
api_key = 'apiKey=4ffqyhUjxE9zcWp0lJDsyI2Jam%2B6nJXYbx%2Bfn2t/6rTqceF0JPWs7gwV5dV39Vqn' # private api key

In [65]:
# Test 
print(default_path + api_type + sep + data_type + sep + date_type + sep + start_date + sep + end_date + sep + station + sep + cnt + sep + page + sep + api_key)

https://data.kma.go.kr/apiData/getData?type=json&dataCd=ASOS&dateCd=DAY&startDt=20160101&endDt=20190430&stnIds=108&schListCnt=999&pageIndex=&apiKey=4ffqyhUjxE9zcWp0lJDsyI2Jam%2B6nJXYbx%2Bfn2t/6rTqceF0JPWs7gwV5dV39Vqn


## Contents

In [61]:
# 2개만 추출
for calen in range(1,3) :
    
    # 서울 2년 4개월 간 날씨 Import
    response = requests.get(default_path + api_type + sep + data_type + sep + date_type + sep + start_date + sep + end_date + sep +
                        station + sep + cnt + sep + page + str(calen) + sep + api_key , verify=False)
    
    # json Import
    seoul_weather = json.loads(response.text)
    seoul_httpcd = seoul_weather[0]['status']
    seoul_status = seoul_weather[1]['msg']
    seoul_station = seoul_weather[2]['stnIds']

    # 원 소스
    # seoul_weather[3]['info'][0]0
    # 처음 API를 호출하는 경우
    if calen == 1 :
        seoul_df = pd.DataFrame(seoul_weather[3]['info'])
        seoul_df['seoul_httpcd'] = seoul_httpcd
        seoul_df['seoul_status'] = seoul_status
        seoul_df['seoul_station'] = seoul_station
    # 처음 API를 호출하는 것이 아닌 경우
    else :
        seoul_df2 = pd.DataFrame(seoul_weather[3]['info'])
        seoul_df2['seoul_httpcd'] = seoul_httpcd
        seoul_df2['seoul_status'] = seoul_status
        seoul_df2['seoul_station'] = seoul_station
        
#         seoul_df = pd.merge(seoul_df, seoul_df2)

C:\Users\HIT\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\HIT\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [64]:
print('Seoul_df1 Rows :', seoul_df.shape[0], 'Columns :', seoul_df.shape[1])
print('Seoul_df2 Rows :', seoul_df2.shape[0], 'Columns :', seoul_df2.shape[1])

Seoul_df1 Rows : 1216 Columns : 66
Seoul_df2 Rows : 217 Columns : 66


In [63]:
# UNION ALL
sql = '''
    select *
      from seoul_df
    union all
    select *
      from seoul_df2
'''
seoul_df = ps.sqldf(sql)
print('Seoul_df Rows :', seoul_df.shape[0], 'Columns :', seoul_df.shape[1])
seoul_df.head()

Seoul_df Rows : 1216 Columns : 66


,AVG_CM10_TE,AVG_CM20_TE,AVG_CM30_TE,AVG_CM5_TE,AVG_LMAC,AVG_M0_5_TE,AVG_M1_0_TE,AVG_M1_5_TE,AVG_M3_0_TE,AVG_M5_0_TE,...,SUM_GSR,SUM_LRG_EV,SUM_RN,SUM_RN_DUR,SUM_SML_EV,SUM_SS_HR,TM,seoul_httpcd,seoul_status,seoul_station
0,0.3,1.5,2.0,0.1,7.0,3.7,7.3,10.0,15.5,17.1,...,4.61,NaN,NaN,NaN,0.6,2.1,2016-01-01,200,success,108
1,2.1,2.3,2.4,2.8,7.1,3.7,7.2,9.9,15.5,17.1,...,4.50,NaN,NaN,NaN,0.3,3.6,2016-01-02,200,success,108
2,4.2,4.0,3.7,4.9,7.5,4.2,7.1,9.7,15.4,17.0,...,4.85,NaN,NaN,NaN,0.3,2.2,2016-01-03,200,success,108
3,3.7,4.4,4.3,3.3,2.5,4.9,7.1,9.6,15.3,17.0,...,9.03,NaN,NaN,NaN,1.7,8.6,2016-01-04,200,success,108
4,1.4,3.0,3.6,-0.1,0.1,4.8,7.2,9.5,15.2,17.0,...,8.22,NaN,NaN,NaN,1.8,7.9,2016-01-05,200,success,108
